# Defensive Asset Allocation with momentum
## Use Index of representative asset classes
#### Market Index ETF, sector etf, bond etf are included
#### Result is driven from 2010 to 2021 market data


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np

from utils.yahoo_stock_data import yahoo_data
from utils.regular_rebalancing import momentum_rebalance_strategy

%matplotlib inline
from matplotlib import pyplot as plt

import quantstats

### Load Data of each asset classes
##### Use representative ETF to model strategy
##### Data should be in yld format [rows : Dates, Columns : Asset Classes, yld = log(currnet_period_price/previous_period_price]

In [ ]:
asset_class_ticker =[
    "LQD",  # iShares iBoxx $ Investment Grade Corporate Bond ETF   2002-07-22
    "DBC",  # Invesco DB Commodity Index Tracking Fund              2006-02-03
    "XLB",  # The Materials Select Sector SPDR® Fund                1998-12-16
    "XLE",  # The Energy Select Sector SPDR® Fund                   1998-12-22
    "XLF",  # The Financial Select Sector SPDR® Fund                1998-12-22
    "XLI",  # The Industrial Select Sector SPDR® Fund               1998-12-22
    "XLK",  # The Technology Select Sector SPDR® Fund               1998-12-22
    "XME",  # SPDR® S&P® Metals & Mining ETF                        2006-06-23
    "XLP",  # The Consumer Staples Select Sector SPDR® Fund         1998-12-22
    "XLY",  # The Consumer Discretionary Select Sector SPDR® Fund   1998-12-22
    "XLU",  # The Utilities Select Sector SPDR® Fund                1998-12-22
    "XLV",  # The Health Care Select Sector SPDR® Fund              1998-12-22
    "XPH",  # SPDR® S&P® Pharmaceuticals ETF                        2006-06-22
    "XBI",  # SPDR® S&P® Biotech ETF                                2006-02-06

    # Defensive Assets
    "SHY",  # iShares 1-3 Year Treasury Bond ETF                    2002-07-22
    "IEI",  # iShares 3-7 Year Treasury Bond ETF                    2007-01-05

    # Canaria Assets
    "VWO",  # Vanguard FTSE Emerging Markets ETF                    2005-03-04
    "BND",  # iShares 7-10 Year Treasury Bond ETF                   2002-07-22
    ]

bm_class_ticker =[
    "VTI", # Vanguard Total Stock Market ETF                        2001-05-24
    "TLT", # iShares 20+ Year Treasury Bond ETF                     2002-07-22
    "IEI", # iShares 3-7 Year Treasury Bond ETF                     2007-01-05
    "GLD", # SPDR Gold Trust                                        2004-11-18
    "GSG"  # iShares S&P GSCI Commodity Indexed Trust               2006-07-10
                    ]

In [ ]:
# for weekly rebalance strategy
data_setting = {
    'start' : '2007-01-01',
    'end' : '2022-03-01',
    'ticker_list': asset_class_ticker,
    'rebal_period' : '1W',
}

data = yahoo_data(**data_setting)
yield_df = data.yahoo_stock_data_loading()
yield_df

# to draw benchmark
data_setting = {
    'start' : '2007-01-01',
    'end' : '2022-03-01',
    'ticker_list': bm_class_ticker,
    'rebal_period' : '1W',
}

data = yahoo_data(**data_setting)
benchmark_yield_df = data.yahoo_stock_data_loading()

In [ ]:
def all_weather_strategy(yld_df):
    weights = pd.DataFrame(columns=yld_df.columns, index=yld_df.index)
    weights.loc[:,'VTI'] = 0.300
    weights.loc[:,'TLT'] = 0.400
    weights.loc[:,'IEI'] = 0.150
    weights.loc[:,'GLD'] = 0.075
    weights.loc[:,'GSG'] = 0.075
    return weights

In [ ]:
def get_momentum(yld_df):
    """
    yiled_df : dataframe with weekly yield of asset classes
    """
    momentum = pd.DataFrame(columns = yld_df.columns, index = yld_df.index)

    for asset in  yld_df.columns:
        i = 0
        for date in yld_df.index:
            # data set consists of weekly data
            # 52 weeks per year = 12 month per year
            if i > 52 :
                # first 12 month data (52 data points) cannot be used since 12 month lagged returns is required
                current = yld_df[asset].iloc[i]
                before_1m = yld_df[asset].iloc[i-4]
                before_3m = yld_df[asset].iloc[i-13]
                before_6m = yld_df[asset].iloc[i-26]
                before_12m = yld_df[asset].iloc[i-52]

                momentum.loc[date, asset] = (current/before_1m - 1) * 12 + (current/before_3m - 1) * 4 \
                                            + (current/before_6m - 1) * 2 + (current/before_12m - 1) * 1
            else:
                momentum.loc[date, asset] = 0
            i = i + 1


    # abnormal data processing
    momentum = momentum.replace([np.inf], 1000)
    momentum = momentum.replace([-np.inf], -1000)
    momentum = momentum.replace([np.nan], 0)

    return momentum

def select_sector(yld_df):
    momentum_df = get_momentum(yld_df)
    selected_momentum = pd.DataFrame(
        columns=['momentum_1','momentum_2'],
        index=momentum_df.index
    )
    selected_ticker = pd.DataFrame(
        columns=['momentum_1','momentum_2'],
        index=momentum_df.index
    )
    selectable_asset = [
    "LQD",  # iShares iBoxx $ Investment Grade Corporate Bond ETF   2002-07-22
    "DBC",  # Invesco DB Commodity Index Tracking Fund              2006-02-03
    "XLB",  # The Materials Select Sector SPDR® Fund                1998-12-16
    "XLE",  # The Energy Select Sector SPDR® Fund                   1998-12-22
    "XLF",  # The Financial Select Sector SPDR® Fund                1998-12-22
    "XLI",  # The Industrial Select Sector SPDR® Fund               1998-12-22
    "XLK",  # The Technology Select Sector SPDR® Fund               1998-12-22
    "XME",  # SPDR® S&P® Metals & Mining ETF                        2006-06-23
    "XLP",  # The Consumer Staples Select Sector SPDR® Fund         1998-12-22
    "XLY",  # The Consumer Discretionary Select Sector SPDR® Fund   1998-12-22
    "XLU",  # The Utilities Select Sector SPDR® Fund                1998-12-22
    "XLV",  # The Health Care Select Sector SPDR® Fund              1998-12-22
    "XPH",  # SPDR® S&P® Pharmaceuticals ETF                        2006-06-22
    "XBI",  # SPDR® S&P® Biotech ETF                                2006-02-06
    ]

    for date in momentum_df.index:
        emerging_momentum = momentum_df.loc[date,'VWO']
        bnd_momentum = momentum_df.loc[date,'BND']

        sorted_momentum = momentum_df.loc[date].sort_values(ascending=False)

        for i in selectable_asset:

            if emerging_momentum >= 0 and bnd_momentum >= 0:
                selected_momentum.loc[date,f'momentum_{i+1}'] = sorted_momentum[i]
                selected_ticker.loc[date,f'momentum_{i+1}'] = sorted_momentum.index[i]
            else:
                selected_momentum.loc[date,f'momentum_{i+1}'] = sorted_momentum[i]
                selected_ticker.loc[date,f'momentum_{i+1}'] = sorted_momentum.index[i]




            if sorted_momentum[i] <= shy_momentum:
                selected_momentum.loc[date,f'momentum_{i+1}'] = shy_momentum
                selected_ticker.loc[date,f'momentum_{i+1}'] = 'SHY'
            elif sorted_momentum[i] >= sp500_momentum:
                selected_momentum.loc[date,f'momentum_{i+1}'] = sorted_momentum[i]
                selected_ticker.loc[date,f'momentum_{i+1}'] = sorted_momentum.index[i]
            # if sector momentum score is below S&P500, and above SHY, choose SP500
            elif sorted_momentum[i] <= sp500_momentum and sorted_momentum[i] >= shy_momentum:
                selected_momentum.loc[date,f'momentum_{i+1}'] = sp500_momentum
                selected_ticker.loc[date,f'momentum_{i+1}'] = 'IVV'
            # if momentum is below zero, escape to SHY
            elif sorted_momentum[i] <= 0 :
                selected_momentum.loc[date,f'momentum_{i+1}'] = shy_momentum
                selected_ticker.loc[date,f'momentum_{i+1}'] = 'SHY'
            else:
                selected_momentum.loc[date,f'momentum_{i+1}'] = shy_momentum
                selected_ticker.loc[date,f'momentum_{i+1}'] = 'SHY'


        for i in range(len(selected_momentum.columns)):
            selected_momentum.loc[date,f'momentum_{i+1}'] = sorted_momentum[i]
            selected_ticker.loc[date,f'momentum_{i+1}'] = sorted_momentum.index[i]

    return selected_ticker


def sector_momentum(yld_df):
    mom_ticker_df = select_sector(yld_df)
    mp_table = pd.DataFrame(columns=yld_df.columns, index=yld_df.index)
    for date in yld_df.index:
        selected = mom_ticker_df.loc[date].tolist()
        for sel in selected:
            mp_table.loc[date, sel] = 1/5
    mp_table = mp_table.fillna(0)

    return mp_table


In [ ]:
mp_table_result = sector_momentum(yield_df)
mp_table_result

In [ ]:
def trim_data(yld_df, mp_table, benchmark_yield_df):

    yld_df = yld_df.iloc[52 + 1:]
    mp_table = mp_table.iloc[52 + 1:]
    benchmark_yield_df = benchmark_yield_df.iloc[52 + 1:]
    return yld_df, mp_table, benchmark_yield_df
yield_df, mp_table_result, benchmark_yield_df = trim_data(yield_df, mp_table_result, benchmark_yield_df)

In [ ]:
mp_table_result

In [ ]:
rebal_strategy = momentum_rebalance_strategy(
    strategy=sector_momentum, yld_df=yield_df,
    bm_strategy=sp500_buy_hold_strategy, bm_yld_df=benchmark_yield_df,
    lookback_period=0
)






